# SLEAP Distance Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys
from collections import defaultdict

In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import sleap.process_pose

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

# Functions

In [8]:
def combine_dicts(dicts):
    """
    Combine lists from multiple dictionaries that share the same key.

    This function takes a list of dictionaries where each dictionary's values are lists of numbers.
    It merges these lists for each corresponding key across all dictionaries, producing a single
    dictionary where each key has a combined list of all numbers from the input dictionaries.

    Parameters:
        dicts (list of dict): A list of dictionaries with values as lists of numbers.

    Returns:
        dict: A dictionary with keys from the input dictionaries and values as merged lists 
        of numbers from all corresponding input dictionary values.
    """
    combined = defaultdict(list)
    for dictionary in dicts:
        for key, value in dictionary.items():
            combined[key].extend(value)
    
    return dict(combined)


# Example usage
list_of_dicts = [
    {'a': [1, 2], 'b': [3, 4]},
    {'a': [5], 'b': [6, 7]},
    {'a': [8, 9], 'c': [10]}
]

combined_dict = combine_dicts(list_of_dicts)
print(combined_dict)

{'a': [1, 2, 5, 8, 9], 'b': [3, 4, 6, 7], 'c': [10]}


In [9]:
def find_consecutive_ranges(numbers, min_length=1):
    """
    Finds the start and end indices for consecutive ranges of each number in a list where the range meets a minimum length.

    This function iterates through a list of integers and identifies ranges where the same integer appears consecutively
    and the length of this sequence meets or exceeds the specified minimum length.

    Parameters:
        numbers (list): A list of integers to analyze for consecutive ranges.
        min_length (int): The minimum length of a range for it to be included in the results.

    Returns:
        dict: A dictionary with integers as keys and a list of tuples (start, end) as values,
              where each tuple represents the start and end indices (inclusive) of consecutive ranges
              for that integer. Only ranges that meet or exceed the minimum length are included.
    """
    ranges = {}
    n = len(numbers)
    if n == 0:
        return ranges
    
    start = 0
    current = numbers[0]

    for i in range(1, n):
        if numbers[i] != current:
            if (i - start) >= min_length:
                if current not in ranges:
                    ranges[current] = []
                ranges[current].append((start, i - 1))
            current = numbers[i]
            start = i

    # Handle the last range
    if (n - start) >= min_length:
        if current not in ranges:
            ranges[current] = []
        ranges[current].append((start, n - 1))

    return ranges

# Example usage:
numbers = [1, 1, 2, 2, 2, 3, 3, 3, 3, 2, 2, 1, 1]
print(find_consecutive_ranges(numbers, min_length=3))

{2: [(2, 4)], 3: [(5, 8)]}


In [10]:
def update_tuples_in_dict(original_dict, reference_list):
    """
    Updates the values in the tuples within a dictionary by replacing indices with corresponding values from a reference list.
    
    This function iterates through each key-value pair in the original dictionary. Each value is expected to be a list of tuples,
    where each tuple contains indices. These indices are used to fetch corresponding values from the reference list, creating new tuples.
    
    Parameters:
        original_dict (dict): Dictionary whose values are lists of tuples. Each tuple consists of indices into the reference_list.
        reference_list (list): List of elements that are referenced by the indices in the tuples of the original_dict.
        
    Returns:
        dict: A dictionary with the same keys as original_dict but with tuples transformed to contain elements from reference_list
              based on the indices in the original tuples.
    """
    # Create a new dictionary to store the updated key-value pairs
    new_dict = {}
    for key, list_of_tuples in original_dict.items():
        # Process each tuple in the list associated with the current key
        updated_tuples = [
            tuple(reference_list[idx] for idx in tup) for tup in list_of_tuples
        ]
        new_dict[key] = updated_tuples
    
    return new_dict

# Example usage:
original_dict = {
    'a': [(0, 1), (2, 3)],
    'b': [(1, 3), (0, 2)]
}
reference_list = ['alpha', 'beta', 'gamma', 'delta']

updated_dict = update_tuples_in_dict(original_dict, reference_list)
print(updated_dict)

{'a': [('alpha', 'beta'), ('gamma', 'delta')], 'b': [('beta', 'delta'), ('alpha', 'gamma')]}


## Inputs & Data

Explanation of each input and where it comes from.

In [11]:
OUTPUT_PREFIX = "rce_pilot_3_alone_comp"

In [12]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX))
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_01_lfp_traces_and_frames.pkl".format(OUTPUT_PREFIX))



In [13]:
# SLEAP_DF = pd.read_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_comp_ext/results/2024_06_26_sleap_clustering/proc/grouped_exploded_columns.pkl")
SLEAP_DF = pd.read_pickle("../2024_06_26_sleap_clustering/proc/grouped_exploded_columns.pkl")

In [14]:
SLEAP_DF.head()

,video_name,current_subject,frame_index,tone_start_frame,tone_stop_frame,condition,competition_closeness,notes,10s_before_tone_frame,10s_after_tone_frame,...,subject_nose,subject_tail_base,agent_thorax,agent_nose,agent_tail_base,session_dir,experiment,sleap_name,video_id,agent
0,20240317_151922_long_comp_subj_3-1_and_3-3.1,3.1,"[1086, 1087, 1088, 1089, 1090, 1091, 1092, 109...","[1286, 1286, 1286, 1286, 1286, 1286, 1286, 128...","[1486, 1486, 1486, 1486, 1486, 1486, 1486, 148...","[3.3, 3.3, 3.3, 3.3, 3.3, 3.3, 3.3, 3.3, 3.3, ...","[Subj 2 then Subj 1, Subj 2 then Subj 1, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1086, 1086, 1086, 1086, 1086, 1086, 1086, 108...","[1686, 1686, 1686, 1686, 1686, 1686, 1686, 168...",...,"[[60.14673983110819, 38.0809500426292], [60.14...","[[65.86894804065032, 31.8436244490543], [65.86...","[[51.04336785027878, 11.609328010205985], [51....","[[43.139588993359475, 13.385259646809214], [43...","[[54.49241922411595, 12.722769593270066], [54....",20240317_151922_long_comp_subj_3-1_and_3-3,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1.f...,5,3.3
1,20240317_151922_long_comp_subj_3-1_and_3-3.1,3.3,"[1086, 1087, 1088, 1089, 1090, 1091, 1092, 109...","[1286, 1286, 1286, 1286, 1286, 1286, 1286, 128...","[1486, 1486, 1486, 1486, 1486, 1486, 1486, 148...","[3.3, 3.3, 3.3, 3.3, 3.3, 3.3, 3.3, 3.3, 3.3, ...","[Subj 2 then Subj 1, Subj 2 then Subj 1, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1086, 1086, 1086, 1086, 1086, 1086, 1086, 108...","[1686, 1686, 1686, 1686, 1686, 1686, 1686, 168...",...,"[[43.139588993359475, 13.385259646809214], [43...","[[54.49241922411595, 12.722769593270066], [54....","[[64.11036004541992, 34.137273635147906], [64....","[[60.14673983110819, 38.0809500426292], [60.14...","[[65.86894804065032, 31.8436244490543], [65.86...",20240317_151922_long_comp_subj_3-1_and_3-3,long_comp,20240317_151922_long_comp_subj_3-1_and_3-3.1.f...,5,3.1
2,20240317_172017_long_comp_subj_4-2_and_4-3.1,4.2,"[1100, 1101, 1102, 1103, 1104, 1105, 1106, 110...","[1300, 1300, 1300, 1300, 1300, 1300, 1300, 130...","[1501, 1501, 1501, 1501, 1501, 1501, 1501, 150...","[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 4.2, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1100, 1100, 1100, 1100, 1100, 1100, 1100, 110...","[1701, 1701, 1701, 1701, 1701, 1701, 1701, 170...",...,"[[65.23110812441777, 8.220414676017084], [62.5...","[[72.8621368084351, 13.53665765315761], [72.41...","[[57.3449526902064, 32.078917344899914], [57.5...","[[60.79884848875681, 35.963595797483414], [60....","[[56.32709801682682, 29.966639024342353], [56....",20240317_172017_long_comp_subj_4-2_and_4-3,long_comp,20240317_172017_long_comp_subj_4-2_and_4-3.1.f...,6,4.3
3,20240317_172017_long_comp_subj_4-2_and_4-3.1,4.3,"[1100, 1101, 1102, 1103, 1104, 1105, 1106, 110...","[1300, 1300, 1300, 1300, 1300, 1300, 1300, 130...","[1501, 1501, 1501, 1501, 1501, 1501, 1501, 150...","[4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 4.2, 4.2, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1100, 1100, 1100, 1100, 1100, 1100, 1100, 110...","[1701, 1701, 1701, 1701, 1701, 1701, 1701, 170...",...,"[[60.79884848875681, 35.963595797483414], [60....","[[56.32709801682682, 29.966639024342353], [56....","[[72.18478668206376, 10.328978616940205], [71....","[[65.23110812441777, 8.220414676017084], [62.5...","[[72.8621368084351, 13.53665765315761], [72.41...",20240317_172017_long_comp_subj_4-2_and_4-3,long_comp,20240317_172017_long_comp_subj_4-2_and_4-3.1.f...,6,4.2
4,20240318_143819_long_comp_subj_3-3_and_3-4.1,3.3,"[1134, 1135, 1136, 1137, 1138, 1139, 1140, 114...","[1334, 1334, 1334, 1334, 1334, 1334, 1334, 133...","[1533, 1533, 1533, 1533, 1533, 1533, 1533, 153...","[temp, temp, temp, temp, temp, temp, temp, tem...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [15]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [16]:
FULL_LFP_TRACES_PKL = "{}_04_spectral_and_sleap.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

## Merging the dataframes based on shared video name

In [17]:
LFP_SPECTRAL_DF["video_name"] = LFP_SPECTRAL_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [18]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
0,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
1,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-1_t6b6_merged,3.1,...,"[nan, 0.07223674, 0.06942491, 0.065466404, 0.0...","[nan, 0.0046885125, 0.0033377786, 0.0018332143...","[0.01945638, nan, nan, nan, 0.019393958, nan, ...","[0.0130572505, nan, nan, nan, 0.007762581, nan...","[nan, nan, 0.03636455, 0.029700823, 0.02418904...","[nan, nan, 0.0018358257, 0.0017288909, 0.00135...","[0.033121496, 0.041370735, 0.055094175, 0.0427...","[0.0017061487, 0.0015902787, 0.0014763202, 0.0...","[0.016354451, 0.025327114, 0.028878175, 0.0222...","[0.00209533, 0.0015905038, 0.0014053435, 0.001..."
2,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 74], [1272, 1471], [3268, 3469], [4466, 4...","[[0, 74], [152, 157], [252, 305], [305, 323], ...","[[0, 74], [160, 225], [254, 257], [257, 279], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.1,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
3,rce_pilot_3,20240320_142408_alone_comp_subj_3-1_and_3-3,"[[0, 75], [1273, 1472], [3269, 3470], [4467, 4...","[[0, 75], [153, 158], [252, 306], [306, 324], ...","[[0, 75], [161, 226], [255, 258], [258, 280], ...",20240320_142408_alone_comp_subj_3-1_and_3-3.2,NaN,/scratch/back_up/reward_competition_extention/...,20240320_142408_alone_comp_subj_3-3_t5b5_merged,3.3,...,"[0.0408452, 0.04246293, 0.04184469, nan, nan, ...","[0.0008264334, 0.0009773864, 0.0009832539, nan...","[0.024847934, 0.047361273, 0.042880192, nan, n...","[0.0006459263, 0.00090438855, 0.0010187915, na...","[0.023980794, 0.04565177, 0.03644787, nan, nan...","[0.00055783533, 0.00084574166, 0.0008706296, n...","[0.016893655, 0.027668845, 0.022960208, nan, n...","[0.0006401466, 0.0010481183, 0.0009893934, nan...","[0.021055607, 0.030588005, 0.023756983, nan, n...","[0.0008363458, 0.001157422, 0.0010688329, nan,..."
4,rce_pilot_3,20240320_171038_alone_comp_subj_4-2_and_4-3,"[[0, 79], [1276, 1477], [3272, 3473], [4470, 4...","[[0, 79], [84, 92], [92, 132]

In [19]:
SLEAP_DF["video_name"] = SLEAP_DF["video_name"].apply(lambda x: x.strip(".videoTimeStamps.cameraHWSync"))


In [20]:
cols_to_use = LFP_SPECTRAL_DF.columns.difference(SLEAP_DF.columns)
merged_LFP_SPECTRAL_DF = pd.merge(SLEAP_DF, LFP_SPECTRAL_DF[list(cols_to_use) + ["video_name", "current_subject"]], on=["video_name", "current_subject"], how="inner")

In [21]:
merged_LFP_SPECTRAL_DF.head()

,video_name,current_subject,frame_index,tone_start_frame,tone_stop_frame,condition,competition_closeness,notes,10s_before_tone_frame,10s_after_tone_frame,...,vHPC_gamma_phase,vHPC_lfp_RMS_filtered,vHPC_lfp_trace,vHPC_mPFC_granger_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,vHPC_power_gamma,vHPC_power_theta,vHPC_theta_band,vHPC_theta_phase,video_timestamps
0,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.2,"[34042, 34043, 34044, 34045, 34046, 34047, 340...","[34242, 34242, 34242, 34242, 34242, 34242, 342...","[34442, 34442, 34442, 34442, 34442, 34442, 344...","[4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, ...","[After trial, After trial, After trial, After ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[34642, 34642, 34642, 34642, 34642, 34642, 346...",...,"[0.023950227, 0.96471804, 1.2889154, 1.6205887...","[-0.375, -0.569, -0.786, -0.974, -1.335, -1.65...","[-74.685, -113.49, -156.78, -194.21999, -266.1...","[[nan, nan, nan, nan, nan, nan, nan, nan, nan,...","[[0.0013803464, 0.0079310145, 0.009366615, 0.0...","[0.00366902, nan, nan, 0.002184509, 0.00216185...","[0.046842232, nan, nan, 0.023685867, 0.0444350...","[-0.37066078, -0.35752764, -0.3432352, -0.3277...","[2.7155704, -2.8761566, -2.806554, -2.548516, ...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831..."
1,20240320_171038_alone_comp_subj_4-2_and_4-3.1,4.3,"[34042, 34043, 34044, 34045, 34046, 34047, 340...","[34242, 34242, 34242, 34242, 34242, 34242, 342...","[34442, 34442, 34442, 34442, 34442, 34442, 344...","[4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, 4.3, ...","[After trial, After trial, After trial, After ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34042, 34042, 34042, 34042, 34042, 34042, 340...","[34642, 34642, 34642, 34642, 34642, 34642, 346...",...,"[2.0565047, 2.3807256, 2.6063833, 2.87997, 3.1...","[0.254, 0.382, 0.505, 0.664, 0.985, 0.99, 0.61...","[53.234997, 80.145, 106.079994, 139.42499, 206...","[[2.7552054, 1.5484817, 2.318468, 0.67259854, ...","[[0.001118467, 0.0013452128, 0.009780567, 0.01...","[0.0017462745, 0.0017710698, 0.0011198351, 0.0...","[0.031059334, 0.026878728, 0.018851878, 0.0271...","[0.49282554, 0.4864071, 0.47833988, 0.46862525...","[-0.7696023, -0.28357738, -0.24996911, 0.00574...","[1384, 2770, 4156, 4156, 5541, 6927, 6927, 831..."
2,20240322_120625_alone_comp_subj_3-3_and_3-4.1,3.3,"[34069, 34070, 34071, 34072, 34073, 34074, 340...","[34269, 34269, 34269, 34269, 34269, 34269, 342...","[34468, 34468, 34468, 34468, 34468, 34468, 344...","[3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34069, 34069, 34069, 34069, 34069, 34069, 340...","[34668, 34668, 34668, 34668, 34668, 34668, 346...",...,"[0.011866312, 0.9828468, 1.330576, 1.6782773, ...","[0.027, -0.017, -0.073, -0.221, -0.433, -0.49,...","[3.5099998, -2.145, -9.36, -28.275, -55.379997...","[[0.009626778, 0.042104572, 0.26073232, 0.0171...","[[0.032633882, 0.034465823, 0.037538365, 0.026...","[0.0021949317, 0.0022116762, 0.0016394572, nan...","[0.03049053, 0.017763153, 0.024036832, nan, na...","[0.20751306, 0.24045162, 0.27310878, 0.3053482...","[-1.2177873, -1.0926981, -1.0485686, -0.963749...","[1384, 1983, 2770, 4156, 5541, 5541, 6927, 831..."
3,20240322_120625_alone_comp_subj_3-3_and_3-4.1,3.4,"[34069, 34070, 34071, 34072, 34073, 34074, 340...","[34269, 34269, 34269, 34269, 34269, 34269, 342...","[34468, 34468, 34468, 34468, 34468, 34468, 344...","[3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, 3.4, ...","[Subj 1 then Subj 2, Subj 1 then Subj 2, Subj ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[34069, 34069, 34069, 34069, 34069, 34069, 340...","[34668, 34668, 34668, 34668, 34668, 34668, 346...",...,"[2.5774395, -3.0006855, -2.8295147, -2.449566,...","[0.032, -0.283, 0.276, 0.578, -0.641, -0.791, ...","[3.705, -32.37, 31.589998, 66.104996, -73.32, ...","[[nan, nan, na

## Getting the timestamps of each cluster frame

In [22]:
merged_LFP_SPECTRAL_DF["frame_index"].iloc[0]

array([34042, 34043, 34044, ..., 61590, 61591, 61592])

In [23]:
merged_LFP_SPECTRAL_DF["video_timestamps"].iloc[0]

array([    1384,     2770,     4156, ..., 64384726, 64384726, 64386112],
      dtype=int32)

In [24]:
merged_LFP_SPECTRAL_DF["cluster_timestamp"] = merged_LFP_SPECTRAL_DF.apply(lambda x: x["video_timestamps"][x["frame_index"]], axis=1)


In [25]:
merged_LFP_SPECTRAL_DF["cluster_timestamp"].head()

0    [34079916, 34081302, 34082688, 34084074, 34084...
1    [34079916, 34081302, 34082688, 34084074, 34084...
2    [34103476, 34104862, 34106248, 34106248, 34107...
3    [34103476, 34104862, 34106248, 34106248, 34107...
4    [34096547, 34097933, 34098977, 34099318, 34100...
Name: cluster_timestamp, dtype: object

- Checking if the cluster timestamp shape matches the actual number of clusters

In [26]:
merged_LFP_SPECTRAL_DF["cluster_timestamp"].apply(lambda x: x.shape).head()

0    (12593,)
1    (12593,)
2    (12591,)
3    (12591,)
4    (12593,)
Name: cluster_timestamp, dtype: object

In [27]:
merged_LFP_SPECTRAL_DF["kmeans_cluster"].head()

0    [2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, ...
1    [2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, ...
2    [4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
3    [4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
4    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, ...
Name: kmeans_cluster, dtype: object

In [28]:
merged_LFP_SPECTRAL_DF["kmeans_cluster"].apply(lambda x: x.shape).head()

0    (12593,)
1    (12593,)
2    (12591,)
3    (12591,)
4    (12593,)
Name: kmeans_cluster, dtype: object

In [29]:
FULL_LFP_TRACES_PKL

'rce_pilot_3_alone_comp_04_spectral_and_sleap.pkl'

In [30]:
merged_LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))


In [31]:
merged_LFP_SPECTRAL_DF.shape

(10, 201)

In [32]:
list(SLEAP_DF.columns)

['video_name',
 'current_subject',
 'frame_index',
 'tone_start_frame',
 'tone_stop_frame',
 'condition ',
 'competition_closeness',
 'notes',
 '10s_before_tone_frame',
 '10s_after_tone_frame',
 'subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'standard_embedding_y',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'subject_tail_base',
 'agent_thorax',
 'agent_nose',
 'agent_tail_base',
 'session_dir',
 'experim

In [33]:
merged_LFP_SPECTRAL_DF["recording"].unique()

array(['20240320_171038_alone_comp_subj_4-2_t6b6_merged',
       '20240320_171038_alone_comp_subj_4-3_t5b5_merged',
       '20240322_120625_alone_comp_subj_3-3_t6b6_merged',
       '20240322_120625_alone_comp_subj_3-4_t5b5_merged',
       '20240322_160946_alone_comp_subj_4-3_t6b6_merged',
       '20240322_160946_alone_comp_subj_4-4_t5b5_merged',
       '20240323_144517_alone_comp_subj_3-1_t5b5_merged',
       '20240323_144517_alone_comp_subj_3-4_t6b6_merged',
       '20240323_165815_alone_comp_subj_4-2_t5b5_merged',
       '20240323_165815_alone_comp_subj_4-4_t6b6_merged'], dtype=object)

In [34]:
merged_LFP_SPECTRAL_DF.columns

Index(['video_name', 'current_subject', 'frame_index', 'tone_start_frame',
       'tone_stop_frame', 'condition ', 'competition_closeness', 'notes',
       '10s_before_tone_frame', '10s_after_tone_frame',
       ...
       'vHPC_lfp_RMS_filtered', 'vHPC_lfp_trace',
       'vHPC_mPFC_granger_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'vHPC_power_gamma',
       'vHPC_power_theta', 'vHPC_theta_band', 'vHPC_theta_phase',
       'video_timestamps', 'cluster_timestamp'],
      dtype='object', length=201)

# Finding the ranges of each cluster

In [35]:
for col in merged_LFP_SPECTRAL_DF.columns:
    print(col)

video_name
current_subject
frame_index
tone_start_frame
tone_stop_frame
condition 
competition_closeness
notes
10s_before_tone_frame
10s_after_tone_frame
subject_thorax_to_agent_thorax
nose_to_reward_port_sum
nose_to_reward_port_diff
thorax_velocity_sum
thorax_velocity_diff
to_reward_port_angle_sum
to_reward_port_angle_diff
subject_nose_to_reward_port
subject_thorax_velocity
subject_to_reward_port_angle
agent_nose_to_reward_port
agent_thorax_velocity
agent_to_reward_port_angle
closebool_subject_nose_to_reward_port
closebool_agent_nose_to_reward_port
movingbool_subject_thorax_velocity
movingbool_agent_thorax_velocity
manual_cluster_id
standard_embedding_x
standard_embedding_y
kmeans_cluster
subject_locations
agent_locations
subject_thorax
subject_nose
subject_tail_base
agent_thorax
agent_nose
agent_tail_base
session_dir
experiment
sleap_name
video_id
agent
BLA-to-LH_granger_gamma
BLA-to-LH_granger_theta
BLA-to-MD_granger_gamma
BLA-to-MD_granger_theta
BLA-to-mPFC_granger_gamma
BLA-to-mPF

In [36]:
raise ValueError()

ValueError: 

In [40]:
LFP_SPECTRAL_DF["video_name"].unique()

array(['20240320_142408_alone_comp_subj_3-1_and_3-3.1',
       '20240320_142408_alone_comp_subj_3-1_and_3-3.2',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.2',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.2',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.2',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.1',
       '20240323_122227_alone_comp_subj_5-2_and_5-3.2',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.2',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.2'], dtype=object)

In [41]:
merged_LFP_SPECTRAL_DF["video_name"].unique()

array(['20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [43]:
SLEAP_DF["video_name"].unique()

array(['20240317_151922_long_comp_subj_3-1_and_3-3.1',
       '20240317_172017_long_comp_subj_4-2_and_4-3.1',
       '20240318_143819_long_comp_subj_3-3_and_3-4.1',
       '20240318_170933_long_comp_subj_4-3_and_4-4.1',
       '20240319_134914_long_comp_subj_3-1_and_3-4.1',
       '20240319_160457_long_comp_subj_4-2_and_4-4.1',
       '20240320_114629_long_comp_subj_5-3_and_5-4.1',
       '20240320_171038_alone_comp_subj_4-2_and_4-3.1',
       '20240321_114851_long_comp_subj_5-2_and_5-3.1',
       '20240322_120625_alone_comp_subj_3-3_and_3-4.1',
       '20240322_160946_alone_comp_subj_4-3_and_4-4.1',
       '20240323_144517_alone_comp_subj_3-1_and_3-4.1',
       '20240323_165815_alone_comp_subj_4-2_and_4-4.1'], dtype=object)

In [42]:
raise ValueError()

ValueError: 

## Separating each frame into its own row

In [ ]:
filter_columns = ['video_name',
 'current_subject',
 'session_dir',
 'sleap_name',
 'video_id',
 'agent',
 'cluster_timestamp',
 'first_timestamp',
 'recording']

In [ ]:
explode_columns = ['frame_index',
 'tone_start_frame',
 'tone_stop_frame',
 'condition ',
 'competition_closeness',
 'notes',
 '10s_before_tone_frame',
 '10s_after_tone_frame',
 'subject_thorax_to_agent_thorax',
 'nose_to_reward_port_sum',
 'nose_to_reward_port_diff',
 'thorax_velocity_sum',
 'thorax_velocity_diff',
 'to_reward_port_angle_sum',
 'to_reward_port_angle_diff',
 'subject_nose_to_reward_port',
 'subject_thorax_velocity',
 'subject_to_reward_port_angle',
 'agent_nose_to_reward_port',
 'agent_thorax_velocity',
 'agent_to_reward_port_angle',
 'closebool_subject_nose_to_reward_port',
 'closebool_agent_nose_to_reward_port',
 'movingbool_subject_thorax_velocity',
 'movingbool_agent_thorax_velocity',
 'manual_cluster_id',
 'standard_embedding_x',
 'standard_embedding_y',
 'kmeans_cluster',
 'subject_locations',
 'agent_locations',
 'subject_thorax',
 'subject_nose',
 'subject_tail_base',
 'agent_thorax',
 'agent_nose',
 'agent_tail_base']

In [ ]:
# selecting columns to simplify
filtered_LFP_SPECTRAL_DF = merged_LFP_SPECTRAL_DF[filter_columns + explode_columns].copy()

In [ ]:
filtered_LFP_SPECTRAL_DF.head()

In [ ]:
# explode_columns = [
#     "cluster_timestamp",
#     "kmeans_cluster",
#     "condition ",
#     "competition_closeness",
#     "tone_start_frame"]

In [ ]:
exploded_LFP_SPECTRAL_DF = filtered_LFP_SPECTRAL_DF.explode(explode_columns).reset_index(drop=True)


In [ ]:
exploded_LFP_SPECTRAL_DF.head()

# Grouping columns by shared video, subject, and tone start(aka grouping by trial)

In [ ]:
explode_columns

In [ ]:
groupby_columns = ["video_name", "current_subject", "tone_start_frame"]

In [ ]:
filter_columns += ["tone_stop_frame",
"condition ",
"competition_closeness",
"notes",
"10s_before_tone_frame",
"10s_after_tone_frame"]

In [ ]:
# Define aggregation dictionary
agg_dict = {col: list for col in explode_columns if col not in groupby_columns and col not in filter_columns}

agg_dict.update({col: 'first' for col in filter_columns if col not in groupby_columns})

# Apply groupby and aggregation
grouped_exploded_columns = exploded_LFP_SPECTRAL_DF.groupby(["video_name", "current_subject", "tone_start_frame"]).agg(agg_dict).reset_index()


In [ ]:
grouped_exploded_columns.head()

## Renaming the trial labels

In [ ]:
grouped_exploded_columns.head()

In [ ]:
comp_closeness_dict = {'Subj 1 blocking Subj 2': "competitive",
'Subj 2 Only': "no_comp",
'Subj 2 blocking Subj 1': "competitive",
'Subj 1 then Subj 2': "competitive", 
'Subj 1 Only': "no_comp",
'Subj 2 then Subj 1': "competitive",
'Close Call': "competitive",
'After trial': "no_comp"}

In [ ]:
grouped_exploded_columns["trial_label"] = grouped_exploded_columns.apply(lambda x: "win" if x["current_subject"] == x["condition "]  else ("lose" if x["agent"] == x["condition "] else "tie"), axis=1)
                                                                        

In [ ]:
grouped_exploded_columns["competition_closeness"] = grouped_exploded_columns["competition_closeness"].map(comp_closeness_dict)

## Getting the ranges of each cluster

- Getting the index range

In [ ]:
grouped_exploded_columns["cluster_index_ranges_dict"] = grouped_exploded_columns["kmeans_cluster"].apply(lambda x: find_consecutive_ranges(x))

In [ ]:
grouped_exploded_columns.head()

- Calculating the times in milliseconds of each cluster frame

In [ ]:
grouped_exploded_columns["cluster_times"] = grouped_exploded_columns.apply(lambda x: (np.array(x["cluster_timestamp"]) - x["first_timestamp"]) // 20, axis=1)


- Updating the index to use cluster times and timestamps based on video frame timestamps list

In [ ]:
grouped_exploded_columns["cluster_times_ranges_dict"] = grouped_exploded_columns.apply(lambda x: update_tuples_in_dict(x["cluster_index_ranges_dict"], x["cluster_times"]), axis=1)

In [ ]:
grouped_exploded_columns["cluster_timestamps_ranges_dict"] = grouped_exploded_columns.apply(lambda x: update_tuples_in_dict(x["cluster_index_ranges_dict"], x["cluster_timestamp"]), axis=1)

- Combining the win and loss label with the cluster

In [ ]:
grouped_exploded_columns["trial_cluster_times_ranges_dict"] = grouped_exploded_columns.apply(lambda x: {"{}_{}".format(x["trial_label"], k): v for k, v in x["cluster_times_ranges_dict"].items()}, axis=1)


In [ ]:
grouped_exploded_columns["trial_cluster_timestamps_ranges_dict"] = grouped_exploded_columns.apply(lambda x: {"{}_{}".format(x["trial_label"], k): v for k, v in x["cluster_timestamps_ranges_dict"].items()}, axis=1)


- Grouping all the rows with the same video and subject together

In [ ]:
list(grouped_exploded_columns.columns)

In [ ]:
explode_columns

In [ ]:
other_explode_columns = ["tone_stop_frame",
                         "condition ",
                         "competition_closeness",
                         "notes",
                         "10s_before_tone_frame",
                         "10s_after_tone_frame",
                        'cluster_index_ranges_dict',
 'cluster_times',
 'cluster_times_ranges_dict',
 'cluster_timestamps_ranges_dict',
 'trial_cluster_times_ranges_dict',
 'trial_cluster_timestamps_ranges_dict']

In [ ]:
filter_columns

In [ ]:
# Define columns to be transformed into numpy arrays


# Define aggregation dictionary
agg_dict = {col: list for col in explode_columns + other_explode_columns if col not in groupby_columns and col != "tone_start_frame"}

agg_dict.update({col: 'first' for col in filter_columns + ["tone_start_frame"] if col not in groupby_columns and col not in other_explode_columns})

# Apply groupby and aggregation
video_grouped_exploded_columns = grouped_exploded_columns.groupby(["video_name", "current_subject"]).agg(agg_dict).reset_index()


In [ ]:
video_grouped_exploded_columns.columns

In [ ]:
video_grouped_exploded_columns.head()

- Combining all the dictionaries together

In [ ]:
for col in [_ for _ in video_grouped_exploded_columns if "dict" in _]:
    video_grouped_exploded_columns[col] = video_grouped_exploded_columns[col].apply(lambda x: combine_dicts(x))

In [ ]:
video_grouped_exploded_columns.head()

In [ ]:
video_grouped_exploded_columns.to_pickle("./proc/{}_cluster_ranges.pkl".format(OUTPUT_PREFIX))

In [ ]:
raise ValueError()

In [ ]:
video_grouped_exploded_columns.columns